In [2]:
import scipy
from pathlib import Path
import pathlib
from scipy.io.wavfile import read
from scipy.signal import hann
from scipy import signal
from scipy.fftpack import rfft
import matplotlib.pyplot as plt
import librosa
import librosa.display
from yaafelib import *

#file_path = '/Users/mcassettix/Music/'
path = pathlib.Path("~/Music/*.mp3").expanduser()
parts = path.parts[1:] if path.is_absolute() else path.parts
output = pathlib.Path(path.root).glob(str(pathlib.Path("").joinpath(*parts)))
#file_name = 'Bassnectar.wav'
#audio_file = file_path + file_name
#X, sr = librosa.load(audio_file)

In [ ]:
#rmse = librosa.feature.rmse(y=y)[0]
#times = librosa.frames_to_time(np.arange(len(rmse)))
#plt.figure(figsize=(12, 4))
#plt.plot(times, rmse)
#plt.axhline(np.mean(rmse), color='r', alpha=0.5)
#plt.xlabel('Time')
#plt.ylabel('RMSE')
#plt.axis('tight')
#plt.tight_layout()
#mfccs,chroma,mel,contrast,tonnetz = other_features(y,sr)
#ext_features = np.hstack([mfccs,chroma,mel,contrast,tonnetz])
def count_rmse_above_mean(rmse):
    lower_count = 0
    upper_count = 0
    for e in rmse:
        if e > np.mean(rmse):
            upper_count+=1
        else:
            lower_count +=1
    return upper_count/lower_count

def count_delta_above_mean(rmse):
    dyn_change = 0
    for i in range(len(rmse)-3):
        diff = abs(rmse[i+3]-rmse[i])
        if diff > np.mean(rmse):
            dyn_change += 1
    return dyn_change  

In [ ]:
def harmonic_ratio(signal):
    ratios = []
    y_harmonic, y_percussive = librosa.effects.hpss(signal)
    if len(y_harmonic)== len(y_percussive):
        for i in range(len(y_harmonic)):
            if y_harmonic[i] > 0 and y_percussive[i] > 0:
                ratios.append(y_harmonic[i]/y_percussive[i])
    else:
        print('lenghts not idenitcal')        
    return np.median(ratios)
    
def extract_features(signal):

    return [
        np.mean(librosa.feature.zero_crossing_rate(signal)),
        np.mean(librosa.feature.rmse(signal)[0]),
        np.max(librosa.feature.rmse(signal)[0]),
        count_rmse_above_mean(librosa.feature.rmse(signal)[0]),
        count_delta_above_mean(librosa.feature.rmse(signal)[0]),
        np.mean(librosa.feature.spectral_flatness(signal)),
        np.mean(librosa.feature.spectral_centroid(signal)),
        
    ]
def other_features(X,sample_rate):
    stft = np.abs(librosa.stft(X))
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X),
    sr=sample_rate).T,axis=0)
    return mfccs,chroma,mel,contrast,tonnetz    
    

In [4]:
for p in output:
    y, sr = librosa.load(p) 
    break
    
    # Run the default beat tracker
tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
print('Estimated tempo: {:.2f} beats per minute'.format(tempo))
features = extract_features(y)
features.append(harmonic_ratio(y))
features.append(tempo)
features.reverse()

/Users/mcassettix/Music/01 - Everyday.mp3
Estimated tempo: 143.55 beats per minute


In [26]:
#Since I have no clue what to do with the time series features, let just
#utilize our computed features above 
y_values = []
print(p)
var = input("Enter 1 or 0, 1 for like, 0 for dislike: ")
print(f"you entered {var}, is this correct?")
correct = input("enter y/n: ")
if correct == 'y':
    features.append(var)
else:
    print('wrong input')

Enter 1 or 0, 1 for like, 0 for dislike: 1
you entered 1, is this correct?
enter y/n: y


In [41]:
import pandas as pd
cols=['bpm','harmonic_rythmic_ratio',\
'spectral_centroid','spectral_flatness','count_delta_above_mean',\
'ratio_above_rmse_mean','max_rmse','mean_rms','mean_zcr','liked']

In [52]:
new_dict = {}
for index,col in cols):
    print(features[index],col)
    new_dict[col] = features[index]
    

143.5546875 bpm
2.696205 harmonic_rythmic_ratio
3456.1472723162724 spectral_centroid
0.068789154 spectral_flatness
7 count_delta_above_mean
1.2368010692804634 ratio_above_rmse_mean
0.5595749 max_rmse
0.23579076 mean_rms
0.16048016608467783 mean_zcr
1 liked
